Copyright 2023-2023 Lawrence Livermore National Security, LLC and other MuyGPyS
Project Developers. See the top-level COPYRIGHT file for details.

SPDX-License-Identifier: MIT

# SOAP Kernel Tutorial

This notebook demonstrates how to use the specialized SOAP (Smooth Overlap of Atomic Positions) kernel.
The form of this kernel is hard-coded to be a dot-product.

⚠️ _Note that this is still an experimental feature._ ⚠️

In [ ]:
# general imports
import numpy as np
import matplotlib.pyplot as plt

# MuyGPyS imports
# Must be on iap_develop branch
from MuyGPyS.gp import MuyGPS
from MuyGPyS.gp.deformation import DifferenceIsotropy, dot
from MuyGPyS.gp.hyperparameter import Parameter
from MuyGPyS.gp.noise import HomoscedasticNoise
from MuyGPyS.gp.kernels.experimental import SOAPKernel
from MuyGPyS.optimize import Bayes_optimize
from MuyGPyS.optimize.loss import lool_fn
from MuyGPyS._test.soap import (
    explicit_crosswise,
    explicit_pairwise
)
from MuyGPyS._test.soap import *

%load_ext autoreload
%autoreload 2

## Data Setup

When setting up the data for this kernel, we assume that the user has an array of all of the relevant descriptors, forces, and derivatives for the test and training sets.
Below, we create a set of random, arbitrary data for demonstration.

In [ ]:
desc_count = 116
train_atom_count = 10
train_env_count = 10
test_atom_count = 7
test_env_count = 7

nn_count = 5

test_indices = np.arange(test_env_count)
test_frames = np.zeros(shape=test_env_count) #which frame each env is in
train_frames = np.zeros(shape=train_env_count) #which frame each env is in

nn_envs = np.array([np.random.choice(train_env_count, size = (nn_count), replace=False) for i in range(test_env_count)],dtype=int) #(for each test env, what are the nearby train envs)


train_forces_raw = np.random.normal(loc=0, scale=1, size=(train_env_count, 3))
test_forces_raw = np.random.normal(loc=0, scale=1, size=(test_env_count, 3)) # don't use but somehow necessary


train_descriptors = np.random.normal(loc=0, scale=1, size=(train_env_count, desc_count))
train_derivs = np.random.normal(loc=0, scale=1, size=(train_env_count, train_atom_count, 3, desc_count))

test_descriptors = np.random.normal(loc=0, scale=1, size=(test_env_count, desc_count))
test_derivs = np.random.normal(loc=0, scale=1, size=(test_env_count, test_atom_count, 3, desc_count))


Going from descriptors to the proper feature tensor shapes can be done with either the utility in the test suite for the SOAP kernel or independently.
The requirement for the shape is as follows:

`features.shape=(train_count or test_count, feature_count, 2, atom_count, desc_count)`.

The purpose of this organization is to hold all of the descriptors and derivatives for every atom/environment together in a clear way.

In [ ]:
(test_features, test_forces) = create_tensors_for_muygps(test_descriptors, test_derivs, test_forces_raw, test_frames)
(train_features, train_forces) = create_tensors_for_muygps(train_descriptors, train_derivs, train_forces_raw, train_frames)

print(test_descriptors.shape, test_derivs.shape)
print(test_features.shape)
print(train_descriptors.shape, train_derivs.shape)
print(train_features.shape)

## Kernel Setup

The form of the SOAP kernel is as follows:
$$ K_{f_i f_j} = \sigma^2 \zeta \sum \left[(\zeta-1)(\hat{q}_n \cdot \hat{q}_m)^{\zeta-2} \left( \frac{\partial \hat{q}_m}{\partial x_j} \cdot \hat{q}_n \right) \left( \frac{\partial \hat{q}_n}{\partial x_i} \cdot \hat{q}_m \right) + (\hat{q}_n \cdot \hat{q}_m)^{\zeta-1} \left( \frac{\partial \hat{q}_m}{\partial x_j} \cdot \frac{\partial \hat{q}_n}{\partial x_i} \right) \right]. $$

In the `MuyGPyS` framework, we need to set up crosswise and pairwise tensors.
To do so, we define "similarities" as an analog to differences in our dot product, but use them as an instance of `Difference Isotropy`.

In [ ]:
# redefine sim fn and kernel
sim_fn = DifferenceIsotropy(
    metric=dot,
    length_scale=Parameter(1.0)
)
soap_kernel_fn = MuyGPS(
    kernel=SOAPKernel(
        deformation=sim_fn,
        sensitivity=Parameter(4.0)
    ),
    noise=HomoscedasticNoise(1e-10),
)

The SOAP kernel uses a method analogous to the other kernels in MuyGPyS to set up the crosswise and pairwise tensors.
The shapes of the tensors should be as follows:
* `pairwise_similarity.shape = (test_count, 3, nn_count, 3, nn_count, 4, train_atom_count, train_atom_count)`
* `crosswise_similarity.shape = (test_count, 3, nn_count, 3, 4, teat_atom_count, train_atom_count)`

In [ ]:
crosswise_similarity = sim_fn.crosswise_tensor(
    data=test_features,
    nn_data=train_features,
    data_indices=test_indices,
    nn_indices=nn_envs
)
print(f'Crosswise tensor shape: {crosswise_similarity.shape}')

In [ ]:
pairwise_similarity = sim_fn.pairwise_tensor(
    data=train_features,
    nn_indices=nn_envs
)#.reshape(7, 3, 10, 3, 10, 4, 10, 10, order='F')
print(f'Pairwise tensor shape: {pairwise_similarity.shape}')

Now we set up the kernel tensors from the pairwise and crosswise tensors.
These will have shapes:
* `Kin.shape = (test_count, 3, nn_count, 3, nn_count)`
* `Kcross.shape = (test_count, 3, nn_count, 3)`

In [ ]:
# pairwise
Kin = soap_kernel_fn.kernel(pairwise_similarity)
print(f'Kin shape: {Kin.shape}')

# crosswise
Kcross = soap_kernel_fn.kernel(crosswise_similarity)
print(f'Kcross shape: {Kcross.shape}')

### Kout

In order to compute a posterior variance, we need to supply a prior covariance term on the test points.
This term needs to be computed separately in the case that we are using inner products and similarity tensors, as opposed to difference tensors.
This can be created by computing an `out_tensor` by calling `Deformation.out_tensor()` followed by `MuyGPS.kernel(out_tensor)`.

* `out_similarity.shape = (test_count, 3, 3, 4, test_atom_count, test_atom_count)`
* `Kout.shape = (test_count, 3, 3)`

In [ ]:
# similarity
out_similarity = sim_fn.out_tensor(
    data=test_features,
    data_indices=test_indices
)

In [ ]:
print(f'out_similarity shape = {out_similarity.shape}')

In [ ]:
# Kout
Kout = soap_kernel_fn.kernel(out_similarity)

In [ ]:
print(f'Kout shape = {Kout.shape}')

## Posterior Predictions

To compute posterior predictions, the user needs to set up a tensor of nearest neighbor targets.
Because of the shape expectation of MuyGPyS, this should have dimensions `(test_count, 3, nn_count)`, hence why we swap the latter axes.

Also note that while we have named arrays "batched", the kernel as defined does not support explicit batching for predictions or optimizations.

The posterior shapes should be:
* `posterior_mean.shape = (test_count, 3)`
* `posterior_variance.shape = (test_count, 3, 3)`

In [ ]:
batch_nn_targets = train_forces[nn_envs].swapaxes(-1, -2)

In [ ]:
muygpys_posterior_mean = soap_kernel_fn.posterior_mean(
    Kin=Kin,
    Kcross=Kcross,
    batch_nn_targets=batch_nn_targets
)

In [ ]:
muygpys_posterior_mean.shape

In [ ]:
muygpys_posterior_variance = soap_kernel_fn.posterior_variance(
    Kin=Kin,
    Kcross=Kcross,
    Kout=Kout
)

In [ ]:
muygpys_posterior_variance.shape

# Optimizer

**NOTE:** The SOAPKernel *only* supports mse optimization.

The following is an outline for optimizer setup.
The data used for this example are randomly generated, so the optimizer is not fully run (otehrwise it will error).

In [ ]:
from MuyGPyS.optimize.batch import sample_batch
from MuyGPyS.optimize import Bayes_optimize
from MuyGPyS.optimize.loss import mse_fn
from MuyGPyS.gp.hyperparameter import AnalyticScale
from MuyGPyS.neighbors import NN_Wrapper

To set up a kernel for hyperparameter optimization, it is the same as the setup above.
However, for a parameter to be optimizable, we must define it like `Parameter(initial_guess, [lower_bound, upper_bound])`.

In [ ]:
soap_kernel_fn_for_opt = MuyGPS(
    kernel=SOAPKernel(
        deformation=sim_fn,
        sensitivity=Parameter(4.0, [2.0, 5.0])
    ),
    noise=HomoscedasticNoise(1e-10),
    scale=AnalyticScale()
)

In the example case where we have small datasets, it is reasonable to do a 1:1 test using all of the indices.
That case also makes it simple to set up the batch indices and nn indices.

In a typical use case, a neighbor list should be supplied for all points into the training set.

In [ ]:
batch_indices = np.arange(train_forces.shape[0])
batch_nn_indices = np.array([np.random.choice(train_env_count, size = (nn_count), replace=False) for i in range(train_env_count)],dtype=int) #(for each train env, what are the nearby train envs)

In [ ]:
batch_targets = train_forces[batch_indices]
batch_nn_targets = train_forces[batch_nn_indices].swapaxes(1, 2)

In [ ]:
batch_nn_targets.shape

Set up the pairwise and crosswise training tensors.

In [ ]:
# need pairwise and crosswise diffs
batch_crosswise_sim = sim_fn.crosswise_tensor(
    train_features,
    train_features,
    batch_indices,
    batch_nn_indices,
)

batch_pairwise_sim = sim_fn.pairwise_tensor(
    train_features, batch_nn_indices
)

In [ ]:
soap_mse_optimized = Bayes_optimize(
    soap_kernel_fn_for_opt,
    batch_targets,
    batch_nn_targets,
    batch_crosswise_sim,
    batch_pairwise_sim,
    loss_fn=mse_fn,
    verbose=True,
    init_points=0,
    n_iter=0,
)